In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [ ]:
DIR = '../input/face-mask-lite-dataset'
CLASSES = ['with_mask', 'without_mask']

In [ ]:
image_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=(0.2, 0.8),
    shear_range=0.2,
    zoom_range=0.2,
    channel_shift_range=0.2,
    fill_mode='nearest',
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1./255,
    validation_split=0.3, 
    dtype=None
)

In [ ]:
train_gen = image_gen.flow_from_directory(
    DIR,
    target_size=(224, 224),
    batch_size=64,
    subset='training',
    shuffle=True,
    classes=CLASSES
)

In [ ]:
validation_gen = image_gen.flow_from_directory(
    DIR,
    target_size=(224, 224),
    batch_size=64,
    subset='validation',
    shuffle=True,
    classes=CLASSES
)

In [ ]:
fig = plt.figure(figsize=(8, 8))
columns = 4
rows = 5
for i in range(1, columns*rows +1):
    img, label = train_gen.next()
    fig.add_subplot(rows, columns, i)
    plt.imshow(img[0])
    img.size
plt.show()

In [ ]:
import numpy as np

# plot function, used to plot images with labels within jupyter notebook
def plots(ims, figsize=(12,6), rows=8, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims= np.array(ims * 255).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

imgs, labels = next(train_gen)

# we plot these samples of images and their labels 1 batch at a time.
plots(imgs, titles=labels)

## Building Fine-tuned VGG16 model

In [ ]:
from tensorflow.keras import applications

In [ ]:
vgg16_model = applications.vgg16.VGG16()

In [ ]:
vgg16_model.summary()

In [ ]:
type(vgg16_model)

#### Not a Sequential model so we need to transform the model into Sequential model.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout

#### We need to take off the last Dense layer because default state it classifies 1000 categories

In [ ]:
model = Sequential()

for layer in vgg16_model.layers[: -1]: # add layers axcept lat dense layer[: -1]
    model.add(layer) # add each layer in vgg16 to Sequential model

In [ ]:
model.summary()

#### Freeze layers - excluding the layers from future training. Other words the weights won't be updated.

### Add last Dense layer for binary classification

In [ ]:
from tensorflow.keras.layers import Dense

In [ ]:
model.add(Dense(2, activation='softmax'))

In [ ]:
model.summary()

### Training

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
from tensorflow.keras import callbacks

In [ ]:
# Reduce learning rate when there is a change lesser than <min_delta> in <val_accuracy> for more than <patience> epochs
reduce_lr = callbacks.ReduceLROnPlateau(monitor = 'val_accuracy',
                                                 mode = 'max',
                                                 min_delta = 0.01,
                                                 patience = 3,
                                                 factor = 0.25,
                                                 verbose = 1,
                                                 cooldown = 0,
                                                 min_lr = 0.00000001)

In [ ]:
# Stop the training process when there is a change lesser than <min_delta> in <val_accuracy> for more than <patience> epochs
early_stopping = callbacks.EarlyStopping(monitor = 'val_accuracy',
                                                 mode = 'max',
                                                 min_delta = 0.005,
                                                 patience = 10,
                                                 verbose = 1,
                                                 restore_best_weights = True)

In [ ]:
hist = model.fit(
    train_gen,
    validation_data=validation_gen,
    epochs=10,
    verbose=1,
    callbacks=[reduce_lr, early_stopping]
)

In [ ]:
import matplotlib.pyplot as plt

def gen_graph(history, title):
    
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()

  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()
  plt.show()

In [ ]:
gen_graph(hist, 'Face Mask Detection')

### Save Model

In [ ]:
model.save('saved_model/vgg_model.h5')

### Predictions

In [ ]:
img_file ='w_m_2.jpg'
def get_processed_input_img(image_path, size=224):
  test_img =  cv2.imread(image_path) 
  test_img = cv2.resize(test_img, dsize=(size, size), interpolation=cv2.INTER_AREA)
 
  test_img = test_img.reshape((1, size, size, 3)).astype(np.float32)
  
  return test_img/255

In [ ]:
image = get_processed_input_img(img_file)

In [ ]:
pred = model.predict(image)
print(pred)

In [ ]:
best_2 = (-pred).argsort()[0]

In [ ]:
print(best_2)

In [ ]:
print([CLASSES[i] for i in best_2])